In [3]:
import json
import openai

#step 1: Define 2 functions: self-defined function and text completions
def get_weather(location, unit="fahrenheit"):
    if "beijing" in location.lower():
        return json.dumps({"location": location, "temperature": "11", "unit": "celsius"})
    elif "tokyo" in location.lower():
        return json.dumps({"location": location, "temperature": "33", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "22", "unit": "celsius"})

def chat_completions(parameter_message):
    
    response = openai.chat.completions.create(
        model    = "gpt-3.5-turbo-1106",
        messages = parameter_message,
        tools    = ai_function,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

    return response.choices[0].message

#step 2: Define parameters for text completion, and functions for transferring to Tools

ai_prompt = [{"role"   : "user",
              "content": "What's the weather in tokyo?"}]

ai_function = [
    {
        "type"    : "function",
        "function": {
            "name": "get_weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": 
                        {"type": "string", 
                         "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location"],
            },
        },
    }
]



In [4]:
# step 3: first round call text completions, to get the response_message/tool_calls
first_response = chat_completions(ai_prompt)
tool_calls = first_response.tool_calls


NameError: name 'response_message' is not defined

In [5]:
print("\ntool_calls --> ", first_response.tool_calls)


tool_calls -->  [ChatCompletionMessageToolCall(id='call_5zh5MOlrkF8R8EsQEyMlfb5n', function=Function(arguments='{"location":"Tokyo","unit":"celsius"}', name='get_weather'), type='function')]


In [6]:
if tool_calls:
        available_functions = {
            "get_weather": get_weather,
        }  # only one function in this example, but you can have multiple
        ai_prompt.append(first_response)  # step 4: extend conversation with assistant's reply
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            ai_prompt.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # step 5: extend conversation with function response
        print("\nfinal msg1 --> ", ai_prompt)
        second_response = chat_completions(ai_prompt)  # get a new response from the model where it can see the function response
        print("\n second_response --> ", second_response)


final msg1 -->  [{'role': 'user', 'content': "What's the weather in tokyo?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5zh5MOlrkF8R8EsQEyMlfb5n', function=Function(arguments='{"location":"Tokyo","unit":"celsius"}', name='get_weather'), type='function')]), {'tool_call_id': 'call_5zh5MOlrkF8R8EsQEyMlfb5n', 'role': 'tool', 'name': 'get_weather', 'content': '{"location": "Tokyo", "temperature": "33", "unit": "celsius"}'}]

 second_response -->  ChatCompletionMessage(content='The weather in Tokyo is 33°C. Enjoy the sunny day!', role='assistant', function_call=None, tool_calls=None)
